# AI Bootcamp 03 - Multi-Agent System

In [1]:
import os
from openai import OpenAI

In [2]:
# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY", "YOUR_API_KEY"))

In [3]:
# ------------------ MEMORY MODULE ------------------
class Memory:
    """Stores past interactions and maintains execution history."""
    def __init__(self):
        self.history = []

    def store(self, entry):
        self.history.append(entry)

    def retrieve(self):
        return "\n".join(self.history[-5:])  # Keep recent 5 interactions

memory = Memory()

# ------------------ USER PROXY AGENT ------------------
class UserProxyAgent:
    """Handles user interactions and forwards requests."""
    def __init__(self, coordinator):
        self.coordinator = coordinator

    def process_user_request(self, request):
        print(f"\n🔹 User Request: {request}")
        response = self.coordinator.handle_request(request)
        return response

# ------------------ RETRIEVAL AGENT ------------------
class RetrievalAgent:
    """Retrieves coding guidelines and relevant information."""
    def __init__(self):
        self.knowledge_base = {
            "Python best practices": "Follow PEP8, use meaningful variable names, write docstrings.",
            "Security guidelines": "Sanitize inputs, validate data, avoid hardcoded credentials.",
            "Unit testing": "Use pytest or unittest, mock external dependencies, ensure coverage."
        }

    def search(self, query):
        """Search for relevant guidelines."""
        for key, value in self.knowledge_base.items():
            if key.lower() in query.lower():
                return value
        return "No relevant guidelines found."

# ------------------ CODING AGENT ------------------
class CodingAgent:
    """Generates Python code based on user request and coding guidelines."""
    def generate_code(self, task_description, guidelines):
        prompt = (
            f"Write Python code for the following task:\n{task_description}\n"
            f"Follow these best practices:\n{guidelines}"
        )
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
            max_tokens=300
        )
        return response.choices[0].message.content.strip()

# ------------------ COORDINATOR AGENT ------------------
class CoordinatorAgent:
    """Manages workflow, assigns tasks to agents, and maintains memory."""
    def __init__(self):
        self.retrieval_agent = RetrievalAgent()
        self.coding_agent = CodingAgent()

    def handle_request(self, request):
        """Handles incoming requests and delegates tasks to agents."""
        memory.store(f"User Request: {request}")
        
        # Step 1: Retrieve best practices
        guidelines = self.retrieval_agent.search(request)
        print(f"\n📌 Retrieved Guidelines: {guidelines}")

        # Step 2: Generate Code
        generated_code = self.coding_agent.generate_code(request, guidelines)
        memory.store(f"Generated Code:\n{generated_code}")

        return generated_code

In [6]:
# ------------------ SYSTEM EXECUTION ------------------
if __name__ == "__main__":
    # Initialize the multi-agent system
    coordinator_agent = CoordinatorAgent()
    user_proxy_agent = UserProxyAgent(coordinator_agent)

    # Example User Request
    user_query = "Write a function in Python to calculate Fibonacci sequence, using Unit Testing."
    
    # Process user request
    output_code = user_proxy_agent.process_user_request(user_query)

    print("\n🔹 Final Generated Code:\n", output_code)


🔹 User Request: Write a function in Python to calculate Fibonacci sequence, using Unit Testing.

📌 Retrieved Guidelines: Use pytest or unittest, mock external dependencies, ensure coverage.

🔹 Final Generated Code:
 Here is an example of a Python function to calculate the Fibonacci sequence using pytest for unit testing:

```python
# fibonacci.py

def fibonacci(n):
    if n <= 0:
        return "Invalid input"
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
```

```python
# test_fibonacci.py

import pytest
from fibonacci import fibonacci

def test_fibonacci():
    assert fibonacci(1) == 0
    assert fibonacci(2) == 1
    assert fibonacci(3) == 1
    assert fibonacci(4) == 2
    assert fibonacci(5) == 3
    assert fibonacci(6) == 5
    assert fibonacci(7) == 8
    assert fibonacci(8) == 13
    assert fibonacci(9) == 21
    assert fibonacci(10) == 34
    assert fibonacci(0) == "Invalid input"

if __name__ == "_